In [104]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bemas-project/train.csv
/kaggle/input/bemas-project/test.csv


In [105]:
train = pd.read_csv("../input/bemas-project/train.csv")
test = pd.read_csv("../input/bemas-project/test.csv")
val = test.iloc[0:100]
test = test.iloc[101:]


train_texts=train["message"].values.tolist()
val_texts = val["message"].values.tolist()
train_labels=train["labels"].values.tolist()
val_labels=val["labels"].values.tolist()
test_texts=test["message"].values.tolist()
test_labels=test["labels"].values.tolist()


l1 = []

for i in train_labels:
    if i=="Yes":
        l1.append(1)
    else:
        l1.append(0)

l2 = []
for i in val_labels:
    if i=="Yes":
        l2.append(1)
    else:
        l2.append(0)
    

l3 = []
for i in test_labels:
    if i=="Yes":
        l3.append(1)
    else:
        l3.append(0)

train_labels = l1.copy()
val_labels = l2.copy()
test_labels = l3.copy()
print(train_texts)

["If it's a chemical burn you are not supposed to apply cling film in case it reacts!", 'And this is what this place is coming too... Warnings on social media on how to deal with a chemical burn!!  Its scarey stuff what a world we are living in 😣', 'Every single person making the choice to do this deserves a Darren award.....as for those brings their kids.....😡', 'Guns may save lives but it can also take lives of  innocent ones if handed to bad people', 'Yes, children, make music!!', "Perhaps the worst BBC article I've ever read. Very disappointing.", 'Sleep tight little soldier xx', "After reading some of the comments here I'm starting to wonder idf anyone actually read the story", 'Haha! Leave the kids alone to have some fun!', 'When will good education be within reach of average citizens and much less expensive?', '£145 a year for this. Glad I have the option not to pay... oh wait!', 'Congrats, you just earned another yourself another beat down from SCOTUS. You might as well assume 

In [106]:
from transformers import DistilBertTokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

loading file https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/0e1bbfda7f63a99bb52e3915dcf10c3c92122b827d92eb2d34ce94ee79ba486c.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/8c8624b8ac8aa99c60c912161f8332de003484428c47906d7ff7eb7f73eecdbb.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf

In [107]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [108]:
import torch

class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, train_labels)
val_dataset = IMDbDataset(val_encodings, val_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)

In [109]:
import evaluate
# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     predictions = np.argmax(logits, axis=-1)
#     return metric.compute(predictions=predictions, references=labels)

In [110]:
# !pip install evaluate

In [111]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments


# training_args = TrainingArguments(
#     output_dir='./results',          # output directory
#     num_train_epochs=3,              # total number of training epochs
#     per_device_train_batch_size=16,  # batch size per device during training
#     per_device_eval_batch_size=64,   # batch size for evaluation
#     warmup_steps=500,                # number of warmup steps for learning rate scheduler
#     weight_decay=0.01,               # strength of weight decay
#     logging_dir='./logs',            # directory for storing logs
#     logging_steps=10,
#     report_to="wandb",
# )

# model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

# trainer = Trainer(
#     model=model,                         # the instantiated 🤗 Transformers model to be trained
#     args=training_args,                  # training arguments, defined above
#     train_dataset=train_dataset,         # training dataset
#     eval_dataset=val_dataset,             # evaluation dataset
#     compute_metrics = compute_metrics,
# )

# trainer.train()
# trainer.evaluate()


In [112]:
# trainer.evaluate()
from transformers import TrainerCallback

class CustomCallback(TrainerCallback):
    
    def __init__(self, trainer) -> None:
        super().__init__()
        self._trainer = trainer
    
    def on_epoch_end(self, args, state, control, **kwargs):
        if control.should_evaluate:
            control_copy = deepcopy(control)
            self._trainer.evaluate(eval_dataset=self._trainer.train_dataset, metric_key_prefix="train")
            return control_copy

In [113]:
from datasets import load_metric

# metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    metrics = ["accuracy", "recall", "precision", "f1"] #List of metrics to return
    metric={}
    for met in metrics:
       metric[met] = load_metric(met)
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    metric_res={}
    for met in metrics:
       metric_res[met]=metric[met].compute(predictions=predictions, references=labels)[met]
    return metric_res

# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     predictions = np.argmax(logits, axis=-1)
#     return metric.compute(predictions=predictions, references=labels)

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")
# training_args = TrainingArguments("test_trainer")
training_args = TrainingArguments("test", evaluation_strategy="no")
trainer       = Trainer(
    model         = model, 
    args          = training_args, 
    train_dataset = train_dataset, 
    eval_dataset  = val_dataset,
    compute_metrics = compute_metrics,
)
trainer.add_callback(CustomCallback(trainer)) 
trainer.train()
trainer.evaluate()

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.20.1",
  "vocab_size": 30522
}

loading weights file https://huggingface.co/distilbert-base-uncased/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/9c169103d7e5a73936dd2b627e42851bec0831212b677c637033ee4bce9

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 100
  Batch size = 8


{'eval_loss': 0.6010436415672302,
 'eval_accuracy': 0.75,
 'eval_recall': 0.6285714285714286,
 'eval_precision': 0.6470588235294118,
 'eval_f1': 0.6376811594202899,
 'eval_runtime': 4.2188,
 'eval_samples_per_second': 23.704,
 'eval_steps_per_second': 3.081,
 'epoch': 3.0}

In [114]:
predictions = trainer.predict(test_dataset)
print(predictions.predictions.shape, predictions.label_ids.shape)

***** Running Prediction *****
  Num examples = 110
  Batch size = 8


(110, 2) (110,)


In [115]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

In [116]:
!pip install evaluate
import evaluate
evaluate.logging.set_verbosity_info()
metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.7363636363636363, 'f1': 0.5671641791044776}